In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import json
import html

In [ ]:
def scrape_user_reviews(movie_id, is_fresh):
    base_url = f"https://www.rottentomatoes.com/napi/movie/{movie_id}/reviews/user"
    last = ''
    reviews = []
    while True:
        url = base_url + '?after=' + last
        response = requests.get(url)
        if response.status_code != 200:
          print('Unable to continue fetching:', movie_id)
          break
        response = response.json()
        reviews += [[movie_id, html.unescape(x['quote']), is_fresh] for x in response['reviews'] if (x['rating'] - 3.5 > 0) == is_fresh]

        if len(reviews) >= 20 or not response['pageInfo']['hasNextPage']:
            break
        last = response['pageInfo']['endCursor']

    return reviews

In [ ]:
def scrape_critics_reviews(movie_id, is_fresh):
    base_url = f"https://www.rottentomatoes.com/napi/movie/{movie_id}/reviews/all"
    last = ''
    reviews = []
    while True:
        url = base_url + '?after=' + last
        response = requests.get(url)
        if response.status_code != 200:
          print('Unable to continue fetching:', movie_id)
          break
        response = response.json()
        reviews += [[movie_id, html.unescape(x['quote']), is_fresh] for x in response['reviews'] if x['isFresh'] == is_fresh]

        if len(reviews) >= 20 or not response['pageInfo']['hasNextPage']:
            break
        last = response['pageInfo']['endCursor']

    return reviews

In [ ]:
f_critics = pd.read_csv('f_critics.csv', index_col=1)
f_audience = pd.read_csv('f_audience.csv', index_col=1)
ids = pd.read_csv('movie_ids.csv', index_col=1)

In [ ]:
ids_critics = ids.loc[f_critics.index.intersection(ids.index)]

In [ ]:
ids_audience = ids.loc[f_audience.index.intersection(ids.index)]

In [ ]:
critics_reviews = []
i = 0
for mid in ids_critics.id.values:
  reviews = scrape_critics_reviews(mid, True)
  if not reviews:
    continue
  critics_reviews += reviews
  i += 1
  if i % 20 == 0:
    print(i)

20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480


In [ ]:
pd.DataFrame(critics_reviews, columns=['id', 'review_content', 'is_fresh']).set_index('id').to_csv('critics_fresh.csv', index='id')

In [ ]:
critics_reviews_a = []
i = 0
for mid in ids_audience.id.values:
  try:
    reviews = scrape_critics_reviews(mid, False)
    if not reviews:
      continue
  except Exception as e:
    print('error', str(e))
    continue
  critics_reviews_a += reviews
  i += 1
  if i % 20 == 0:
    print(i)

20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480


In [ ]:
pd.DataFrame(critics_reviews_a, columns=['id', 'review_content', 'is_fresh']).set_index('id').to_csv('critics_rotten.csv', index='id')

In [ ]:
audience_reviews1 = []
i = 0
for mid in ids_audience.id.values:
  try:
    reviews = scrape_user_reviews(mid, True)
    if not reviews:
      continue
  except Exception as e:
    print('error', mid, str(e))
  audience_reviews1 += reviews
  i += 1
  if i % 20 == 0:
    print(i)

20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480


In [ ]:
pd.DataFrame(audience_reviews1, columns=['id', 'review_content', 'is_fresh']).set_index('id').to_csv('audience_fresh.csv', index='id')

In [ ]:
audience_reviews2 = []
i = 0
for mid in ids_critics.id.values:
  try:
    reviews = scrape_user_reviews(mid, False)
    if not reviews:
      continue
  except Exception as e:
    print('error', mid, str(e))
  audience_reviews2 += reviews
  i += 1
  if i % 20 == 0:
    print(i)

20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480


In [ ]:
pd.DataFrame(audience_reviews2, columns=['id', 'review_content', 'is_fresh']).set_index('id').to_csv('audience_rotten.csv', index='id')